In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# Changing working directory to a new path
os.chdir('/Users/huzefa/IITM-Pravartak-Ai-Ml-Ds-Python/Data-Imputation')

In [3]:
# Verify the change
print("Current working directory:", os.getcwd())

Current working directory: /Users/huzefa/IITM-Pravartak-Ai-Ml-Ds-Python/Data-Imputation


In [4]:
#Importing Data
data=pd.read_csv('../DataSets/GTPvar.csv',index_col=0)

In [5]:
#Checking missing values columnwise
data.isnull().sum(axis=1)

1      0
2      0
3      0
4      0
5      3
      ..
96     0
97     5
98     0
99     0
100    0
Length: 100, dtype: int64

In [6]:
#Creating new variable NApresent to contain all the rows' missing values sum
data['NApresent']=data.isnull().sum(axis=1)

In [7]:
#Extracting rows with no null values
df=data[data.NApresent==0]

In [8]:
#Dropping NApresent column from df
df=df.drop('NApresent',axis=1)

In [9]:
#Converting np to numpy array
df_mat=df.to_numpy()

In [10]:
#Rank of obtained matrix
np.linalg.matrix_rank(df_mat)

5

In [11]:
#SVD decomposition
v,s,u=np.linalg.svd(df_mat.T)

In [12]:
#Setting tolerance
tol=1e-8

In [13]:
#Removing columns that are lesser than the tolerance
rank=min(df_mat.shape)-np.abs(s)[::-1].searchsorted(tol)

In [14]:
#Choosing the null space relation
A=v[:,rank:]
A=A.T
print(A)

[[ 0.04739523 -0.51877539  0.51908496 -0.66965893  0.10360266]
 [ 0.25290867 -0.74555191 -0.49550644  0.25256441  0.26622198]
 [-0.69130741  0.06619155 -0.47253226 -0.41182815  0.3533038 ]]


In [15]:
#Perfoming transformations to obtain and impute possible missing values
len(data)
len(A)  

3

In [16]:
for i in range(0,len(data)):
    if((data.iloc[i,5]==0) | (data.iloc[i,5]>len(A))):
        continue
    else:
        eqnsneeded=data.iloc[i,5]
        aID=np.empty(0,dtype='int64')
        bID=np.empty(0,dtype='int64')
        for j in range(len(data.columns)-1):
            if(pd.isnull(data.iloc[i,j])):
                aID=np.append(aID,j)
            else: bID=np.append(bID,j)
        a=A[0:eqnsneeded,aID]
        a=np.array(a)
        x1=((data.iloc[i,bID].to_numpy()))
        b2=-A[0:eqnsneeded,bID]
        b=np.dot(b2,x1)
        x=np.linalg.solve(a,b)
        data.iloc[i,aID]=x